# ============================================================
# ISyE 6420 - Spring 2025 - Project
# Author: Eromonsele Okojie
# Date:   04/20/2025
# ============================================================

In [37]:
import pandas as pd
import numpy as np
import pymc as pm
import arviz as az
import pytensor.tensor as pt
import matplotlib.pyplot as plt

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 0. Reproducibility & Data Load
# ─────────────────────────────────────────────────────────────────────────────
np.random.seed(42)
df = pd.read_csv("../data/data.csv", sep=";")
df.columns = df.columns.str.strip()
df["Target"] = df["Target"].str.strip()
outcome_map = {"Dropout": 0, "Enrolled": 1, "Graduate": 2}
df["y"] = df["Target"].map(outcome_map)
df["prog_idx"], programs = pd.factorize(df["Course"])
n_programs = len(programs)

# ─────────────────────────────────────────────────────────────────────────────
# 1. Preprocess Features
# ─────────────────────────────────────────────────────────────────────────────
num_cols = [
    "Admission grade",
    "Age at enrollment",
    "Curricular units 1st sem (grade)",
    "Curricular units 2nd sem (grade)",
    "Unemployment rate",
    "Inflation rate",
    "GDP",
]
stds = df[num_cols].std().replace(0, 1)
X_num = (df[num_cols] - df[num_cols].mean()) / stds

cat_cols = ["Gender", "Scholarship holder"]
X_cat = pd.get_dummies(df[cat_cols].astype(str), drop_first=True)

X = np.hstack([X_num.values, X_cat.values])
n_students, n_preds = X.shape

adm_idx = num_cols.index("Admission grade")
adm_z = X[:, adm_idx]
other_idx = np.delete(np.arange(n_preds), adm_idx)

prog_mean_adm = df.groupby("prog_idx")["Admission grade"].mean().values
prog_mean_z = (prog_mean_adm - prog_mean_adm.mean()) / prog_mean_adm.std()
g_idx = df["prog_idx"].values

# ─────────────────────────────────────────────────────────────────────────────
# 2. Build & Sample Models
# ─────────────────────────────────────────────────────────────────────────────
coords = {"obs": np.arange(n_students)}

# 2a) Hierarchical Multinomial Model
with pm.Model(coords=coords) as hier_model:
    # hyperpriors
    alpha0  = pm.Normal("alpha0", 0.0, 1.0)
    alpha1  = pm.Normal("alpha1", 0.0, 1.0)
    sigma_a = pm.HalfNormal("sigma_a", 1.0)
    beta0   = pm.Normal("beta0", 0.0, 1.0)
    beta1   = pm.Normal("beta1", 0.0, 1.0)
    sigma_b = pm.HalfNormal("sigma_b", 1.0)

    a_prog_raw = pm.Normal("a_prog_raw", 0.0, 1.0, shape=(n_programs, 2))
    mu_a       = alpha0 + alpha1 * prog_mean_z[:, None]
    a_prog     = pm.Deterministic("a_prog", mu_a + sigma_a * a_prog_raw)

    b_prog_raw = pm.Normal("b_prog_raw", 0.0, 1.0, shape=(n_programs, 2))
    mu_b       = beta0 + beta1 * prog_mean_z[:, None]
    b_prog_adm = pm.Deterministic("b_prog_adm", mu_b + sigma_b * b_prog_raw)

    b_other    = pm.Normal("b_other", 0.0, 1.0, shape=(other_idx.size, 2))

    a0 = pm.Normal("a0", mu=[-1.0, +0.5], sigma=1.0, shape=2)

    η_nb = (
        a0[None, :]
        + a_prog[g_idx]
        + adm_z[:, None] * b_prog_adm[g_idx]
        + pt.dot(X[:, other_idx], b_other)
    )
    η = pt.concatenate([pt.zeros((n_students, 1)), η_nb], axis=1)
    p = pm.math.softmax(η, axis=1)

    pm.Categorical("y_obs", p=p, observed=df["y"].values, dims="obs")

    hier_trace = pm.sample(
        draws=20, tune=20, init="adapt_diag",
        target_accept=0.95,
        return_inferencedata=True,
        idata_kwargs={"log_likelihood": True},
    )

# draw posterior predictive samples and merge
with hier_model:
    ppc_hier = pm.sample_posterior_predictive(
        hier_trace,
        var_names=["y_obs"],
        random_seed=42,
        return_inferencedata=True
    )
hier_trace.add_groups(posterior_predictive=ppc_hier.posterior_predictive)

# 2b) Flat Multinomial Model (no hierarchy)
with pm.Model(coords=coords) as flat_model:
    a0_flat = pm.Normal("a0_flat", mu=0.0, sigma=2.0, shape=2)
    b_flat  = pm.Normal("b_flat", mu=0.0, sigma=1.0, shape=(n_preds, 2))

    η_nb_flat = a0_flat[None, :] + pt.dot(X, b_flat)
    η_flat    = pt.concatenate([pt.zeros((n_students, 1)), η_nb_flat], axis=1)
    p_flat    = pm.math.softmax(η_flat, axis=1)

    pm.Categorical("y_obs", p=p_flat, observed=df["y"].values, dims="obs")

    flat_trace = pm.sample(
        draws=20, tune=20, init="adapt_diag",
        target_accept=0.95,
        return_inferencedata=True,
        idata_kwargs={"log_likelihood": True},
    )

# ─────────────────────────────────────────────────────────────────────────────
# 3. Diagnostics & Model Comparison
# ─────────────────────────────────────────────────────────────────────────────

# 3.1 Convergence diagnostics
az.plot_trace(
    hier_trace,
    var_names=["alpha0","alpha1","beta0","beta1","sigma_a","sigma_b"]
)
plt.tight_layout()

hier_summary = az.summary(
    hier_trace,
    var_names=["alpha0","alpha1","beta0","beta1","sigma_a","sigma_b","a0","b_other"],
    round_to=2
)
print(hier_summary)

# 3.2 Posterior predictive checks
az.plot_ppc(
    hier_trace,
    group="posterior",
    data_pairs={"y_obs":"y_obs"}
)
plt.show()

# 3.3 LOO comparison
hier_loo = az.loo(hier_trace, scale="deviance")
flat_loo = az.loo(flat_trace, scale="deviance")
cmp = az.compare({"hierarchical": hier_loo, "flat": flat_loo})
print(cmp)

# ─────────────────────────────────────────────────────────────────────────────
# 4. Deeper Interpretation
# ─────────────────────────────────────────────────────────────────────────────
prog_effects = az.summary(
    hier_trace,
    var_names=["a_prog","b_prog_adm"],
    hdi_prob=0.95
)
if 'chain' in prog_effects.index.names:
    print(prog_effects.loc[prog_effects.index.get_level_values("chain")==0].head(10))
else:
    print(prog_effects.head(10))

# Predictive uncertainty for examples
examples = [10, 100, 1000]
posterior_probs = hier_trace.posterior_predictive["y_obs"].stack(sample=("chain","draw")).values
for i, idx in enumerate(examples):
    # Ensure idx is within the bounds of posterior_probs
    idx = min(idx, posterior_probs.shape[1] -1)  
    sims = posterior_probs[:, idx]
    counts = np.bincount(sims, minlength=3) / sims.size
    ci = np.percentile((sims==2).astype(float), [2.5, 97.5])
    print(f"Student {examples[i]}: proportions = {counts.round(2)}, grad CI = {ci.round(2)}")

plt.figure()
az.plot_forest(hier_trace, var_names=["b_prog_adm"], combined=True)
plt.title("Forest of program-level admission slopes")
plt.show()

# ─────────────────────────────────────────────────────────────────────────────
# 5. Novel Extensions (random slopes on unemployment)
# ─────────────────────────────────────────────────────────────────────────────
with pm.Model(coords=coords) as extended_model:
    alpha0_e  = pm.Normal("alpha0_e", 0.0, 1.0)
    alpha1_e  = pm.Normal("alpha1_e", 0.0, 1.0)
    sigma_a_e = pm.HalfNormal("sigma_a_e", 1.0)
    beta0_e   = pm.Normal("beta0_e", 0.0, 1.0)
    beta1_e   = pm.Normal("beta1_e", 0.0, 1.0)
    sigma_b_e = pm.HalfNormal("sigma_b_e", 1.0)

    a_prog_raw_e = pm.Normal("a_prog_raw_e", 0.0, 1.0, shape=(n_programs, 2))
    mu_a_e       = alpha0_e + alpha1_e * prog_mean_z[:, None]
    a_prog_e     = mu_a_e + sigma_a_e * a_prog_raw_e

    b_prog_raw_adm_e = pm.Normal("b_prog_raw_adm_e", 0.0, 1.0, shape=(n_programs, 2))
    mu_b_adm_e       = beta0_e + beta1_e * prog_mean_z[:, None]
    b_prog_adm_e     = mu_b_adm_e + sigma_b_e * b_prog_raw_adm_e

    ur_idx = num_cols.index("Unemployment rate")
    ur_z   = X[:, ur_idx]
    gamma0 = pm.Normal("gamma0", 0.0, 1.0)
    gamma1 = pm.Normal("gamma1", 0.0, 1.0)
    sigma_g= pm.HalfNormal("sigma_g", 1.0)
    g_prog_raw = pm.Normal("g_prog_raw", 0.0, 1.0, shape=(n_programs,2))
    mu_g       = gamma0 + gamma1 * prog_mean_z[:, None]
    b_prog_ur  = pm.Deterministic("b_prog_ur", mu_g + sigma_g * g_prog_raw)

    b_fix_e = pm.Normal("b_fix_e", 0.0, 1.0, shape=(other_idx.size,2))
    a0_e = pm.Normal("a0_e", mu=[-1.0,+0.5], sigma=1.0, shape=2)

    η_nb_e = (
        a0_e[None,:]
        + a_prog_e[g_idx]
        + adm_z[:,None] * b_prog_adm_e[g_idx]
        + ur_z[:,None] * b_prog_ur[g_idx]
        + pt.dot(X[:, other_idx], b_fix_e)
    )
    η_e = pt.concatenate([pt.zeros((n_students,1)), η_nb_e], axis=1)
    p_e = pm.math.softmax(η_e, axis=1)

    pm.Categorical("y_obs", p=p_e, observed=df["y"].values, dims="obs")
    ext_trace = pm.sample(
        draws=20, tune=20, init="adapt_diag", target_accept=0.95,
        return_inferencedata=True, idata_kwargs={"log_likelihood": True}
    )

ext_loo = az.loo(ext_trace, scale="deviance")
az.compare({"flat": flat_loo, "hier": hier_loo, "extended": ext_loo})

# ─────────────────────────────────────────────────────────────────────────────
# 6. Save Results
# ─────────────────────────────────────────────────────────────────────────────
az.to_netcdf(
    hier_trace,
    "hierarchical_model_results.nc",
    group="posterior",
    coords=coords
)
az.to_netcdf(
    flat_trace,
    "flat_model_results.nc",
    group="posterior",
    coords=coords
)
az.to_netcdf(
    ext_trace,
    "extended_model_results.nc",
    group="posterior",
    coords=coords
)
az.to_netcdf(
    hier_trace,
    "hierarchical_model_results_full.nc",
    group="posterior_predictive",
    coords=coords
)
az.to_netcdf(
    flat_trace,
    "flat_model_results_full.nc",
    group="posterior_predictive",
    coords=coords
)
az.to_netcdf(
    ext_trace,
    "extended_model_results_full.nc",
    group="posterior_predictive",
    coords=coords
)

# ─────────────────────────────────────────────────────────────────────────────
# 7. End of Script
# ─────────────────────────────────────────────────────────────────────────────

point={'alpha0': array(0.), 'alpha1': array(0.), 'sigma_a_log__': array(0.), 'beta0': array(0.), 'beta1': array(0.), 'sigma_b_log__': array(0.), 'a_prog_raw': array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]]), 'b_prog_raw': array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]]), 'b_other': array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]]), 'a0': array([-1. ,  0.5])}

No problems found
joint init log‑prob = -4658.329848703395


Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha0, alpha1, sigma_a, beta0, beta1, sigma_b, a_prog_raw, b_prog_raw, b_other, a0]


KeyboardInterrupt: 